# Install and import libraries

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.coma/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.coma/huggingface/accelerate.git /tmp/pip-req-build-75b9a8jm did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.coma/huggingface/accelerate.git /tmp/pip-req-build-75b9a8jm did not run su

In [ ]:
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Load and examine the data

Data description: https://github.com/salesforce/WikiSQL

In [ ]:
dataset = load_dataset("wikisql", trust_remote_code=True)
dataset

Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 15878
    })
    validation: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 8421
    })
    train: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 56355
    })
})

The 'wikisql' dataset has been split into 'train', 'validation', and 'test' dataset.

In [ ]:
# data structure
example = dataset["train"][0]
example

{'phase': 1,
 'question': 'Tell me what the notes are for South Australia ',
 'table': {'header': ['State/territory',
   'Text/background colour',
   'Format',
   'Current slogan',
   'Current series',
   'Notes'],
  'page_title': '',
  'page_id': '',
  'types': ['text', 'text', 'text', 'text', 'text', 'text'],
  'id': '1-1000181-1',
  'section_title': '',
  'caption': '',
  'rows': [['Australian Capital Territory',
    'blue/white',
    'Yaa·nna',
    'ACT · CELEBRATION OF A CENTURY 2013',
    'YIL·00A',
    'Slogan screenprinted on plate'],
   ['New South Wales',
    'black/yellow',
    'aa·nn·aa',
    'NEW SOUTH WALES',
    'BX·99·HI',
    'No slogan on current series'],
   ['New South Wales',
    'black/white',
    'aaa·nna',
    'NSW',
    'CPX·12A',
    'Optional white slimline series'],
   ['Northern Territory',
    'ochre/white',
    'Ca·nn·aa',
    'NT · OUTBACK AUSTRALIA',
    'CB·06·ZZ',
    'New series began in June 2011'],
   ['Queensland',
    'maroon/white',
    'nnn·aaa

By examining the data structure, we may use 'question', 'table(header, rows, caption)' as model input, and 'sql(headable)' as the label.

# Preprocess

In [ ]:
model_name = "google-t5/t5-small"
# instantiate tokenizer of google t5
tokenizer = AutoTokenizer.from_pretrained(model_name)

# test the functionality of the tokenizer
tokenized_example = tokenizer(example['question'])

for key in tokenized_example:
    print(key)
    print(tokenized_example[key])

input_ids
[8779, 140, 125, 8, 3358, 33, 21, 1013, 2051, 1]
attention_mask
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# define a func to preprocess the data
def preprocess_function(examples):
    # Combine question and table information into a single input string
    inputs = []
    for question, table in zip(examples['question'], examples['table']):
        table_str = " | ".join([f"{header} ({type})" for header, type in zip(table['header'], table['types'])])
        input_str = f"Convert the Question to SQL: {question}, based on the table: {table_str}"
        inputs.append(input_str)

    targets = [sql['human_readable'] for sql in examples['sql']]

    return inputs, targets

# test the functionality of preprocess_function
input, target = preprocess_function(dataset['train'])
print(input[0])
print(target[0])

Convert the Question to SQL: Tell me what the notes are for South Australia , based on the table: State/territory (text) | Text/background colour (text) | Format (text) | Current slogan (text) | Current series (text) | Notes (text)
SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA


In [ ]:
# finding the max length for tokenization
tokenizer.model_max_length

512

In [ ]:
# instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# define a func to preprocess the data and tokenize
def preprocess_function(examples):
    # Combine question and table information into a single input string
    inputs = []
    for question, table in zip(examples['question'], examples['table']):
        table_str = " | ".join([f"{header} ({type})" for header, type in zip(table['header'], table['types'])])
        input_str = f"Convert the Question to SQL: {question}, based on the table: {table_str}"
        inputs.append(input_str)

    targets = [sql['human_readable'] for sql in examples['sql']]

    model_inputs = tokenizer(inputs, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
    labels = tokenizer(targets, padding='max_length', max_length=128, truncation=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# preprocess the data and tokenize
train_set = dataset['train'].map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)
validation_set = dataset['validation'].map(preprocess_function, batched=True, remove_columns=dataset['validation'].column_names)
test_set = dataset['test'].map(preprocess_function, batched=True, remove_columns=dataset['test'].column_names)

Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

Map:   0%|          | 0/8421 [00:00<?, ? examples/s]

Map:   0%|          | 0/15878 [00:00<?, ? examples/s]

# Load Google T5 and determine the baseline

The pre-trained model I choose is T5-small version (about 6 million parameters) of Google's T5. Google's T5, short for 'Text-To-Text-Transfer-Transformer', is a pre-trained models based on the Transformer architecture developed by the Google Research Team.

I first evaluate the model without fine-tuning to determine the baseline

Evaluating the generated query quality needs a lot of engineering. Ideally, the quality should be evaluated using execution accuracy by executing the generated queries in actual database or logical form accuracy by calculating how many generated queries are exactly the same as actual queries as illustrated in https://github.com/salesforce/WikiSQL?tab=readme-ov-file


My objective is to analyze the application and potential risks of text-to-SQL LLM in business scenarios, so for simplicity I used BLEU (Bilingual Evaluation Understudy) as the metric, which measures the  n-gram overlap between generated queries and actual queries. However, a single natural language query may correspond to multiple equivalent queries. For example, the order of fields and tables may differ while maintaining the same semantics. The BLEU metric may score these equivalent SQL queries lower because it only considers surface form matching.

In [ ]:
# Load the BLEU metric
bleu_metric = load_metric("bleu")

def compute_metrics(pred):
    # Retrive predicted tokens
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # Decode the tokens and convert to text
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # Compute BLEU score
    bleu = bleu_metric.compute(predictions=[p.split() for p in pred_str], references=[[l.split()] for l in label_str])

    return {"bleu": bleu["bleu"]}

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name, padding=True, return_tensors='pt')

DataCollatorForSeq2Seq dynamically pads the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_metric

model_name = "google-t5/t5-small"
# instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

training_args = Seq2SeqTrainingArguments(
    output_dir="./baseline_results",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

{'eval_loss': 12.96939468383789,
 'eval_bleu': 0.014515387204036716,
 'eval_runtime': 536.6284,
 'eval_samples_per_second': 29.588,
 'eval_steps_per_second': 1.85}

In [ ]:
import pandas as pd

# Select some samples
sample_range = range(5)
test_examples = test_set.select(sample_range)

# Make predictions
pred = trainer.predict(test_examples)
# Retrive predicted tokens
pred_ids = pred.predictions

# Decode the prediction
pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

# Create a DataFrame
questions = [example['question'] for example in dataset['test'].select(sample_range)]
header = [example['table']['header'] for example in dataset['test'].select(sample_range)]
labels = [example['sql']['human_readable'] for example in dataset['test'].select(sample_range)]

df = pd.DataFrame({
    "Question": questions,
    "Header": header,
    "Prediction": pred_str,
    "Target": labels
})

# Show all columns & rows
pd.set_option('display.max_colwidth', None)
df.head()

,Question,Header,Prediction,Target
0,What is terrence ross' nationality,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]",": What is terrence ross' nationality, based on the table:",SELECT Nationality FROM table WHERE Player = Terrence Ross
1,What clu was in toronto 1995-96,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]",": What clu was in toronto 1995-96, based on the table",SELECT School/Club Team FROM table WHERE Years in Toronto = 1995-96
2,which club was in toronto 2003-06,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]",": which club was in toronto 2003-06, based on the table:",SELECT School/Club Team FROM table WHERE Years in Toronto = 2003-06
3,how many schools or teams had jalen rose,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]","SQL: how many schools or teams had jalen rose, according to table: Player",SELECT COUNT School/Club Team FROM table WHERE Player = Jalen Rose
4,Where was Assen held?,"[No, Date, Round, Circuit, Pole Position, Fastest Lap, Race winner, Report]","SQL: Where was Assen held?, based on the table: No (real",SELECT Round FROM table WHERE Circuit = Assen


Without fine-tuning, even when I explicitly ask the model to "Convert the Question to SQL: {question}, based on the table: {table_str}", T5 outputs hallucinations rather than SQL queries. This may because the training data of T5 did not include tabular question answering using SQL.

# Add LoRA Adapter

Now, I want to fine-tune the model so that it learns to generate SQL in its embeddings.

Fine-tuning the entire LLM is beyond the computational powers of a personal device. However, LoRA (Low-Rank Adaptation) significantly reduces the trainable parameters by decompsing some weight matrices in the pre-trained model into the product of 2 low-rank matrices. For instance, we know that in Transformer, each attention head contains three matrices: $W_Q$, $W_K$, and $W_V$, LoRA decomposes each of these matrices into two low-rank matrices such that $W_Q=A_QB_Q$ and updates only $A_Q$ and $B_Q$ during fine-tuning

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


After applying LoRA, I only need to train less than half of the origin parameters, which is about 3 millions. Below shows the model struture, and we can verify that LoRA has been sucessfully added to the model.

In [ ]:
model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=512, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=512, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
               

# Fine-tune

In [ ]:
# hyperparameters in the training_args are cited from https://github.com/anyuanay/medium/blob/main/src/working_huggingface/Working_with_HuggingFace_ch3_Fine_Tuning_T5_Small_Text_Summarization_Model.ipynb
training_args = Seq2SeqTrainingArguments(
    output_dir="my_fine_tuned_t5_small_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=validation_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,0.201800,0.132348,0.480064
2,0.136700,0.095944,0.553002
3,0.120700,0.084832,0.574947
4,0.112300,0.081770,0.582535


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1201: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1201: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1201: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1201: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=14092, training_loss=0.3852757130567912, metrics={'train_runtime': 6018.4652, 'train_samples_per_second': 37.455, 'train_steps_per_second': 2.341, 'total_flos': 3.071297256947712e+16, 'train_loss': 0.3852757130567912, 'epoch': 4.0})

 Both the training and validation loss steadily decrease during training, and in the last epoch, the training loss is slightly better than the validation loss, indicating a good fit. Moreover, the test loss is 0.0833 and the test BLEU is 0.5754, indicating a good generalization ability. Compared to the baseline loss (12.9694) and BLEU (0.0145), fine-tuned T5 is now significantly better at translating text into SQL queries.

In [ ]:
# mount gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Save the model
model.save_pretrained("/content/drive/MyDrive/BA890/olaf_fine_tined_t5_small_model_0")
tokenizer.save_pretrained("/content/drive/MyDrive/BA890/olaf_fine_tined_t5_small_model_0")

('/content/drive/MyDrive/BA890/olaf_fine_tined_t5_small_model_0/tokenizer_config.json',
 '/content/drive/MyDrive/BA890/olaf_fine_tined_t5_small_model_0/special_tokens_map.json',
 '/content/drive/MyDrive/BA890/olaf_fine_tined_t5_small_model_0/spiece.model',
 '/content/drive/MyDrive/BA890/olaf_fine_tined_t5_small_model_0/added_tokens.json',
 '/content/drive/MyDrive/BA890/olaf_fine_tined_t5_small_model_0/tokenizer.json')

# Test

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_metric

# load fine-tuned model
model_path = '/content/drive/MyDrive/BA890/olaf_fine_tined_t5_small_model_0'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.08332476764917374,
 'eval_bleu': 0.5753848907963923,
 'eval_runtime': 615.0969,
 'eval_samples_per_second': 25.814,
 'eval_steps_per_second': 1.614}

the test loss is 0.0833 and the test BLEU is 0.5754, indicating a good generalization ability. Compared to the baseline loss (12.9694) and BLEU (0.0145), fine-tuned T5 is now significantly better at translating text into SQL queries. However,some common mistakes are made by the model as shown below:

In [ ]:
import pandas as pd

# Select some samples
sample_range = range(2000, 3000)
test_examples = test_set.select(sample_range)

# Make predictions
pred = trainer.predict(test_examples)
# Retrive predicted tokens
pred_ids = pred.predictions

# Decode the prediction
pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

# Create a DataFrame
questions = [example['question'] for example in dataset['test'].select(sample_range)]
header = [example['table']['header'] for example in dataset['test'].select(sample_range)]
labels = [example['sql']['human_readable'] for example in dataset['test'].select(sample_range)]

df = pd.DataFrame({
    "Question": questions,
    "Header": header,
    "Prediction": pred_str,
    "Target": labels
})

# Show all columns & rows
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
df.head(100)

,Question,Header,Prediction,Target
0,What is the black caribbean population when the other black population is 2243?,"[Rank, London Borough, Black African Population, Black Caribbean Population, Other Black Population, Total Black Population]",SELECT Black Caribbean Population FROM table WHERE Other Black Population = 2243,SELECT MAX Black Caribbean Population FROM table WHERE Other Black Population = 2243
1,What is the black caribbean population when the black African population is less than 10552.0?,"[Rank, London Borough, Black African Population, Black Caribbean Population, Other Black Population, Total Black Population]",SELECT Black Caribbean Population FROM table WHERE Black Caribbean Population = 10552.0,SELECT MIN Black Caribbean Population FROM table WHERE Black African Population < 10552.0
2,What time slot had an adult rating of 0.6?,"[Season, Timeslot, Season premiere, Season finale, TV season, Rank, U.S. viewers (in millions), Rating (adults 18–49)]",SELECT Time slot FROM table WHERE Rating = 0.6,SELECT Timeslot FROM table WHERE Rating (adults 18–49) = 0.6
3,What season finale date has 2.02 million u.s. Viewers?,"[Season, Timeslot, Season premiere, Season finale, TV season, Rank, U.S. viewers (in millions), Rating (adults 18–49)]",SELECT Season finale FROM table WHERE U.S. viewers = 2.02 million,SELECT Season finale FROM table WHERE U.S. viewers (in millions) = 2.02
4,How many million u.s. Viewers watched season 1?,"[Season, Timeslot, Season premiere, Season finale, TV season, Rank, U.S. viewers (in millions), Rating (adults 18–49)]",SELECT MIN U.S. viewers (in millions) FROM table WHERE,SELECT U.S. viewers (in millions) FROM table WHERE Season = 1
5,How many tv series had an adult rating of 1.2?,"[Season, Timeslot, Season premiere, Season finale, TV season, Rank, U.S. viewers (in millions), Rating (adults 18–49)]",SELECT MIN TV season FROM table WHERE Adult rating = 1.2,SELECT COUNT TV season FROM table WHERE Rating (adults 18–49) = 1.2
6,Where is the University that is also called Hawks?,"[Institution, Location, Founded, Affiliation, Enrollment, Year Joined, Nickname, Conference]",SELECT MIN University FROM table WHERE Affiliation = Hawks,SELECT Location FROM table WHERE Nickname = Hawks
7,Where is the University that is also called Owls?,"[Institution, Location, Founded, Affiliation, Enrollment, Year Joined, Nickname, Conference]",SELECT University FROM table WHERE Affiliation = Owls,SELECT Location FROM table WHERE Nickname = Owls
8,How many University founded in 1863?,"[Institution, Location, Founded, Affiliation, Enrollment, Year Joined, Nickname, Conference]",SELECT Founded FROM table WHERE Institution = 1863,SELECT COUNT Enrollment FROM table WHERE Founded = 1863
9,Where is the University that plays in the American Athletic Conference?,"[Institution, Location, Founded, Affiliation, Enrollment, Year Joined, Nickname, Conference]",SELECT Conference FROM table WHERE Institution = american athletic conference,SELECT Location FROM table WHERE Conference = American Athletic Conference


In the 1rst row, the ‘MIN’ aggregation operator may be unnecessary but the model used the wrong logical operator ‘=’ instead of ‘<’ since ‘<’ rarely appears in the labels. In the 5th row, the model used the wrong aggregation operator and misspelled the header’s name, which might be fixed by fine-tuning hyperparameters such as increasing the training epochs. In the 51rst row, the model failed to use ‘AND’ to connect condition 1 and condition 2 since most labels contain a single condition.

# Future Work
Future work to refine the model includes: 1. Tune the model using datasets with more complex query labels that incorporate window functions, subqueries, CTEs (Common Table Expression), table joins, etc. 2. Add detailed schema and data description to model inputs. 3. Use LLMs with higher capacity 4. Experiment with different combinations of hyper-parameters. 5. Use better evaluation metrics such as execution accuracy and logical form accuracy.